# Check form of function
Here we compare the form with the error function with a numerical convolution.

Although there is reasonable agreement for good time resolution, the proposed form with the error function breaks down when gamma gets large. 

In [1]:
import numpy as np
from scipy.special import erf
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, FloatSlider
from IPython.display import display

%matplotlib widget

In [2]:
%matplotlib widget

# Evaluate original form

In the email, the suggested form is  
$I(t) = I_0/2 + \frac{1}{2} (1+\text{erf}(t))(A*\exp(-t/t_{mag})+C(1-\exp(-t/t_{mag})) + B*\sin…)$

I think $I_0$ is actually more appropriate than $I_0/2$.

The functional behavior is clear if you take the $t_{erf} << t_{mag}$ limit.  

* $-A$ is the drop 
* $-C$ is the amount of order that fails to recover  

We think this should be the same as 

In [3]:
t = np.linspace(-20, 100, 50000)

def exp(x):
    y = np.exp(x)
    y[y>1] = 1
    return y

def revised_form(t, I_0, Gamma, A, C, t_mag):
    I = I_0 + 0.5*(1+erf(2*t*np.log(2)/Gamma))*(A*exp(-t/t_mag)+C*(1-exp(-t/t_mag)))
    return I


def convolution_form(t, Gamma, A, C, t_mag):
    dt = np.mean(np.abs(np.diff(t)))
    x = np.arange(-Gamma*5, Gamma*5+dt, dt)
    t_extended = np.arange(t.min() - 5*Gamma, t.max() + 5*Gamma, dt) 
    kernel = np.exp(-(2*np.sqrt(np.log(2))*x/Gamma)**2)
    kernel = kernel / np.sum(kernel)
    y = (1 - A*np.heaviside(-t_extended, 1) + A*exp(-t_extended/t_mag)
         + C*(1-exp(-t_extended/t_mag))
        )
    I_extended = np.convolve(y, kernel, mode='same')
    return np.interp(t, t_extended, I_extended)


def plot_revised_form(I_0=1., Gamma=.15, A=0.5, C=0.2, t_mag=5):
    I = revised_form(t, I_0=I_0, Gamma=Gamma, A=A, C=C, t_mag=t_mag)
    art1.set_ydata(I)
    I = convolution_form(t, Gamma=Gamma, A=A, C=C, t_mag=t_mag)
    art2.set_ydata(I)
    fig.canvas.draw()

fig, ax = plt.subplots()
I = revised_form(t, I_0=1, Gamma=.01, A=-.8, C=.5, t_mag=5)
art1,  = ax.plot(t, I, label='error function form')
I = convolution_form(t, Gamma=.01, A=-.8, C=.5, t_mag=5)
art2,  = ax.plot(t, I, label='convolution function form')
ax.set_xlabel('t')
ax.set_ylabel('I')
ax.set_ylim((-4, 10))
ax.legend()

def update(I=1):
    art.set_ydata(t*0 + I)
    fig.canvas.draw()

interact(plot_revised_form,
         I0=FloatSlider(value=1, min=0, max=2),
         Gamma=FloatSlider(value=0.15, min=0.00001, max=10, step=0.0001),
         A=FloatSlider(value=-.9, min=-5, max=1),
         C=FloatSlider(value=-.2, min=-5, max=1),
         t_mag=FloatSlider(value=.5, min=0.0001, max=5, step=0.0001),
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='I_0', max=3.0, min=-1.0), FloatSlider(value=0.15, de…

<function __main__.plot_revised_form(I_0=1.0, Gamma=0.15, A=0.5, C=0.2, t_mag=5)>